In [2]:
import cv2
import numpy as np
import os
import json
import ast

In [3]:
# Make a list of all folders in a folder
def get_folder_list(folder):
    return [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

In [4]:
def get_BB(folder, set_list):
    bbox_dict = {}
    for s in set_list:
        seq = cv2.VideoCapture(f"{folder}/{s}/Image#%04d.jpg")
        if not seq.isOpened():
            print('error reading image sequence')
            break
        valid, frame = seq.read()
        if not valid:
            print('error reading image')
            break
        bbox = cv2.selectROIs("Select Rois",frame)
        bbox = [tuple(x) for x in bbox]
        bbox_dict[s] = bbox

    cv2.destroyAllWindows()
    return bbox_dict

In [5]:
def track(folder, tracker, bbox, output_file):
    '''
    Track an object initially denoted by a bounding box (bbox)
    and write out the location in an output file
    '''
    seq = cv2.VideoCapture(f"{folder}/Image#%04d.jpg")
    if not seq.isOpened():
        print('error reading image sequence')
    valid, frame = seq.read()
    if not valid:
        print('error reading image')

    # Initialize tracker with first frame and bounding box
    valid = tracker.init(frame, bbox)

    data = []
    while True:
        # Read a new frame
        valid, frame = seq.read()
        if not valid:
            break

        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        valid, bbox = tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

        # Draw bounding box
        if valid:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        else:
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        x = [timer]
        x.extend(bbox)
        # data
        data.append(np.array(x))
        # Display result
        cv2.imshow("Tracking", frame)

        # Exit if ESC pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):  # if press SPACE bar
            break
    
    # Save data
    np.savetxt(output_file, data, delimiter=',')
    cv2.destroyAllWindows()

In [6]:
def dict_to_json(dict, filename):
    '''Saves a dictionary to a json file'''
    with open(filename, 'w') as f:
        json.dump(dict, f)

def json_to_dict(filename):
    '''Loads a json file into a dictionary'''
    with open(filename) as f:
        return json.load(f)


In [7]:
def dict_to_txt(dict, filename):
    '''Saves a dictionary to a txt file'''
    with open(filename, 'w') as f:
        for key in dict:
            f.write(f"{key}:{dict[key]}\n")

def txt_to_dict(filename):
    '''Converts a txt file into a dictionary'''
    with open(filename) as f:
        return {line.split(':')[0]: ast.literal_eval(line.split(':')[1][:-1]) for line in f}

# Track the Blobs

In [8]:
# Pre-processing
set_list = get_folder_list('data')
bbox_dict = get_BB('data', set_list)
dict_to_txt(bbox_dict, 'bbox_dict.txt')

In [9]:
# Tracking
bbox_dict = txt_to_dict('bbox_dict.txt')
tracker = cv2.TrackerKCF_create()

for s in set_list:
    i = 0
    for bbox in bbox_dict[s]:
        print(s, i, bbox)
        tracker = cv2.TrackerKCF_create()
        output = track(f"data/{s}", tracker, bbox, f"output/{s}-{i}.txt")
        i += 1

Set10a 0 (426, 171, 254, 243)
Set10a 1 (312, 438, 81, 76)
Set10a 2 (0, 532, 43, 57)
Set10a 3 (940, 74, 84, 79)
Set10a 4 (817, 314, 75, 70)
Set10a 5 (831, 616, 84, 94)
Set1a 0 (244, 82, 203, 163)
Set1a 1 (756, 214, 52, 57)
Set1a 2 (1062, 175, 61, 61)
Set1a 3 (866, 364, 110, 117)
Set1a 4 (899, 539, 108, 78)
set1b 0 (378, 200, 119, 102)
set1b 1 (669, 343, 93, 79)
set1b 2 (660, 497, 66, 73)
set1b 3 (447, 547, 180, 153)
set1b 4 (45, 536, 101, 81)
set1b 5 (1149, 394, 109, 105)
Set2a 0 (597, 160, 130, 125)
Set2a 1 (460, 364, 82, 77)
Set2a 2 (96, 538, 63, 78)
Set2a 3 (880, 623, 127, 110)
set2b 0 (290, 309, 123, 126)
set2b 1 (654, 315, 70, 84)
set2b 2 (1083, 214, 73, 80)
set2b 3 (1017, 498, 62, 76)
Set3a 0 (576, 209, 74, 78)
Set3a 1 (455, 519, 54, 67)
Set3a 2 (889, 642, 132, 127)
set3b 0 (381, 288, 66, 81)
set3b 1 (191, 555, 170, 110)
set3b 2 (678, 46, 149, 125)
set3b 3 (887, 412, 84, 80)
Set4a 0 (667, 203, 75, 70)
Set4a 1 (975, 237, 65, 64)
Set4a 2 (1108, 688, 64, 80)
set4b 0 (537, 77, 87, 70)